In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [ ]:
import os
if not os.path.isdir('saved_models_{}'.format('wgan')):
    os.mkdir('saved_models_{}'.format('wgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('wgan'), mode='w')
import cv2
import torch.utils.data as Data

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

import keras.backend as K

import matplotlib.pyplot as plt
import os
import sys

import numpy as np


class WGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        self.clip_value = 0.01
        optimizer = RMSprop(lr=0.00005)

        # Build and compile the critic
        self.critic = self.build_critic()
        self.critic.compile(loss=self.wasserstein_loss,
                            optimizer=optimizer,
                            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.critic.trainable = False

        # The critic takes generated images as input and determines validity
        valid = self.critic(img)

        # The combined model  (stacked generator and critic)
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.wasserstein_loss,
                              optimizer=optimizer,
                              metrics=['accuracy'])

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (X_test, y_test) = fashion_mnist.load_data()

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        X_test = np.expand_dims(X_test, axis=3)
        # y_trai/n = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                for _ in range(self.n_critic):
                    global_step += 1
                    imgs = X_train[index * batch_size:(index + 1) * batch_size]
                    # ---------------------
                    #  Train Discriminator
                    # ---------------------

                    # Select a random batch of images
                    # idx = np.random.randint(0, X_train.shape[0], batch_size)
                    # imgs = X_train[idx]

                    # Sample noise as generator input
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                    # Generate a batch of new images
                    gen_imgs = self.generator.predict(noise)

                    # Train the critic
                    d_loss_real = self.critic.train_on_batch(imgs, valid)
                    d_loss_fake = self.critic.train_on_batch(gen_imgs, fake)
                    d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

                    # Clip critic weights
                    for l in self.critic.layers:
                        weights = l.get_weights()
                        weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                        l.set_weights(weights)

                    # ---------------------
                    #  Train Generator
                    # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d[D loss: %f] [G loss: %f]" % (epoch, global_step, 1 - d_loss[0], 1 - g_loss[0]))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')

if __name__ == '__main__':
    wgan = WGAN()
    wgan.train(epochs=40, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 16)        160       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 32)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 32)          128       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 32)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:5[D loss: 0.999912] [G loss: 1.000173]
epoch:0 step:10[D loss: 0.999925] [G loss: 1.000180]
epoch:0 step:15[D loss: 0.999923] [G loss: 1.000182]
epoch:0 step:20[D loss: 0.999923] [G loss: 1.000173]
epoch:0 step:25[D loss: 0.999924] [G loss: 1.000170]
epoch:0 step:30[D loss: 0.999922] [G loss: 1.000171]
epoch:0 step:35[D loss: 0.999924] [G loss: 1.000177]
epoch:0 step:40[D loss: 0.999927] [G loss: 1.000176]
epoch:0 step:45[D loss: 0.999928] [G loss: 1.000172]
epoch:0 step:50[D loss: 0.999919] [G loss: 1.000171]
epoch:0 step:55[D loss: 0.999922] [G loss: 1.000169]
epoch:0 step:60[D loss: 0.999922] [G loss: 1.000164]
epoch:0 step:65[D loss: 0.999926] [G loss: 1.000155]
epoch:0 step:70[D loss: 0.999924] [G loss: 1.000152]
epoch:0 step:75[D loss: 0.999929] [G loss: 1.000156]
epoch:0 step:80[D loss: 0.999929] [G loss: 1.000135]
epoch:0 step:85[D loss: 0.999932] [G loss: 1.000132]
epoch:0 step:90[D loss: 0.999938] [G loss: 1.000124]
epoch:0 step:95[D loss: 0.999941] [G loss: 1.00

epoch:0 step:700[D loss: 0.999974] [G loss: 1.000065]
epoch:0 step:705[D loss: 0.999977] [G loss: 1.000059]
epoch:0 step:710[D loss: 0.999972] [G loss: 1.000054]
epoch:0 step:715[D loss: 0.999973] [G loss: 1.000061]
epoch:0 step:720[D loss: 0.999976] [G loss: 1.000061]
epoch:0 step:725[D loss: 0.999975] [G loss: 1.000055]
epoch:0 step:730[D loss: 0.999974] [G loss: 1.000063]
epoch:0 step:735[D loss: 0.999976] [G loss: 1.000058]
epoch:0 step:740[D loss: 0.999972] [G loss: 1.000063]
epoch:0 step:745[D loss: 0.999978] [G loss: 1.000054]
epoch:0 step:750[D loss: 0.999975] [G loss: 1.000068]
epoch:0 step:755[D loss: 0.999976] [G loss: 1.000058]
epoch:0 step:760[D loss: 0.999972] [G loss: 1.000059]
epoch:0 step:765[D loss: 0.999974] [G loss: 1.000065]
epoch:0 step:770[D loss: 0.999974] [G loss: 1.000064]
epoch:0 step:775[D loss: 0.999971] [G loss: 1.000065]
epoch:0 step:780[D loss: 0.999976] [G loss: 1.000072]
epoch:0 step:785[D loss: 0.999975] [G loss: 1.000070]
epoch:0 step:790[D loss: 0.9

...

epoch:38 step:181680[D loss: 1.000028] [G loss: 1.000032]
epoch:38 step:181685[D loss: 0.999974] [G loss: 0.999996]
epoch:38 step:181690[D loss: 0.999982] [G loss: 1.000080]
epoch:38 step:181695[D loss: 0.999968] [G loss: 1.000057]
epoch:38 step:181700[D loss: 0.999977] [G loss: 1.000056]
epoch:38 step:181705[D loss: 0.999953] [G loss: 1.000074]
epoch:38 step:181710[D loss: 1.000015] [G loss: 0.999998]
epoch:38 step:181715[D loss: 1.000147] [G loss: 0.999801]
epoch:38 step:181720[D loss: 0.999951] [G loss: 1.000071]
epoch:38 step:181725[D loss: 1.000254] [G loss: 0.999843]
epoch:38 step:181730[D loss: 0.999968] [G loss: 1.000140]
epoch:38 step:181735[D loss: 0.999928] [G loss: 1.000111]
epoch:38 step:181740[D loss: 1.000136] [G loss: 0.999843]
epoch:38 step:181745[D loss: 0.999870] [G loss: 1.000059]
epoch:38 step:181750[D loss: 0.999995] [G loss: 1.000002]
epoch:38 step:181755[D loss: 0.999954] [G loss: 1.000059]
epoch:38 step:181760[D loss: 0.999963] [G loss: 1.000078]
epoch:38 step:

epoch:38 step:182355[D loss: 1.000053] [G loss: 1.000029]
epoch:38 step:182360[D loss: 0.999935] [G loss: 1.000124]
epoch:38 step:182365[D loss: 1.000067] [G loss: 1.000046]
epoch:38 step:182370[D loss: 0.999906] [G loss: 1.000262]
epoch:38 step:182375[D loss: 0.999922] [G loss: 1.000214]
epoch:38 step:182380[D loss: 1.000029] [G loss: 1.000212]
epoch:38 step:182385[D loss: 0.999996] [G loss: 1.000038]
epoch:38 step:182390[D loss: 1.000001] [G loss: 1.000083]
epoch:38 step:182395[D loss: 1.000020] [G loss: 1.000096]
epoch:38 step:182400[D loss: 0.999973] [G loss: 1.000081]
##############
[2.65140751 2.32009436 2.2469022  3.75173279 1.63734518 6.50686597
 2.37355037 3.80781235 4.05611537 5.79330756]
##########
epoch:38 step:182405[D loss: 0.999943] [G loss: 1.000121]
epoch:38 step:182410[D loss: 1.000009] [G loss: 0.999992]
epoch:38 step:182415[D loss: 0.999970] [G loss: 1.000095]
epoch:38 step:182420[D loss: 1.000036] [G loss: 1.000015]
epoch:38 step:182425[D loss: 0.999914] [G loss: 1

epoch:39 step:183020[D loss: 0.999973] [G loss: 1.000072]
epoch:39 step:183025[D loss: 0.999985] [G loss: 1.000111]
epoch:39 step:183030[D loss: 1.000028] [G loss: 1.000039]
epoch:39 step:183035[D loss: 1.000042] [G loss: 1.000030]
epoch:39 step:183040[D loss: 1.000032] [G loss: 0.999941]
epoch:39 step:183045[D loss: 1.000248] [G loss: 0.999626]
epoch:39 step:183050[D loss: 1.000008] [G loss: 0.999956]
epoch:39 step:183055[D loss: 1.000050] [G loss: 0.999957]
epoch:39 step:183060[D loss: 1.000162] [G loss: 1.000161]
epoch:39 step:183065[D loss: 0.999957] [G loss: 1.000148]
epoch:39 step:183070[D loss: 0.999953] [G loss: 1.000206]
epoch:39 step:183075[D loss: 0.999899] [G loss: 1.000396]
epoch:39 step:183080[D loss: 0.999807] [G loss: 1.000263]
epoch:39 step:183085[D loss: 0.999930] [G loss: 1.000057]
epoch:39 step:183090[D loss: 1.000046] [G loss: 0.999971]
epoch:39 step:183095[D loss: 1.000003] [G loss: 0.999933]
epoch:39 step:183100[D loss: 0.999994] [G loss: 0.999987]
epoch:39 step:

epoch:39 step:183700[D loss: 1.000084] [G loss: 1.000020]
epoch:39 step:183705[D loss: 1.000251] [G loss: 0.999897]
epoch:39 step:183710[D loss: 0.999998] [G loss: 1.000188]
epoch:39 step:183715[D loss: 1.000059] [G loss: 1.000230]
epoch:39 step:183720[D loss: 0.999853] [G loss: 1.000305]
epoch:39 step:183725[D loss: 0.999941] [G loss: 1.000140]
epoch:39 step:183730[D loss: 0.999979] [G loss: 1.000087]
epoch:39 step:183735[D loss: 0.999998] [G loss: 1.000026]
epoch:39 step:183740[D loss: 1.000018] [G loss: 1.000014]
epoch:39 step:183745[D loss: 1.000026] [G loss: 0.999962]
epoch:39 step:183750[D loss: 1.000183] [G loss: 0.999678]
epoch:39 step:183755[D loss: 0.999916] [G loss: 0.999899]
epoch:39 step:183760[D loss: 1.000186] [G loss: 0.999820]
epoch:39 step:183765[D loss: 0.999970] [G loss: 1.000034]
epoch:39 step:183770[D loss: 1.000012] [G loss: 1.000032]
epoch:39 step:183775[D loss: 0.999929] [G loss: 1.000059]
epoch:39 step:183780[D loss: 1.000044] [G loss: 0.999995]
epoch:39 step:

epoch:39 step:184375[D loss: 0.999900] [G loss: 1.000308]
epoch:39 step:184380[D loss: 0.999913] [G loss: 1.000261]
epoch:39 step:184385[D loss: 0.999989] [G loss: 1.000012]
epoch:39 step:184390[D loss: 0.999940] [G loss: 1.000087]
epoch:39 step:184395[D loss: 1.000097] [G loss: 0.999930]
epoch:39 step:184400[D loss: 1.000052] [G loss: 0.999885]
##############
[2.56900616 2.33057439 2.27479883 3.5988541  1.56660881 6.95091946
 2.13442934 3.72222162 3.95243739 4.79967577]
##########
epoch:39 step:184405[D loss: 1.000022] [G loss: 0.999789]
epoch:39 step:184410[D loss: 0.999953] [G loss: 1.000089]
epoch:39 step:184415[D loss: 0.999935] [G loss: 1.000068]
epoch:39 step:184420[D loss: 0.999946] [G loss: 1.000123]
epoch:39 step:184425[D loss: 0.999997] [G loss: 0.999992]
epoch:39 step:184430[D loss: 0.999949] [G loss: 1.000115]
epoch:39 step:184435[D loss: 0.999976] [G loss: 1.000071]
epoch:39 step:184440[D loss: 0.999975] [G loss: 1.000126]
epoch:39 step:184445[D loss: 1.000050] [G loss: 1